# 使用 qianfan sdk 构建本地评估模型



千帆大模型平台提供了在线进行自动评估的方式，而有时我们希望有更灵活的自动评估方式。

本文将以评估文本摘要效果为例子来介绍

1. 如何使用千帆 sdk 封装自定义本地评估方法
2. 使用函数封装手搓其原理实现。

以下是本文封装部分使用的库与作用
    Dataset：提供数据集接口，用于构造待评估数据集，方便在本地、平台、huggingface多端拉取数据集。
    Prompt：提供数据模板，用于指导模型进行评估，可联网获取prompt或者使用prompt优化功能。
    Service：提供模型服务接口，在evaluateManager中为没有生成答案的数据集提供生成能力。
    EvaluateManager：提供评估任务管理功能，封装了异步调用算子，可以方便地实现多线程评估。

此处使用0.3.0版本以上的qianfan sdk。

In [ ]:
! pip install -U "qianfan[dataset_base]>=0.3.0"

进行鉴权认证。

In [128]:
import os

os.environ['QIANFAN_ACCESS_KEY'] = 'your_access_key'
os.environ['QIANFAN_SECRET_KEY'] = 'your_secret_key'

# 使用 Service 对象进行评估时，请按实际情况填写 QPS LIMIT，
# 取值为所有 Service QPS Limit中的最小值
os.environ["QIANFAN_QPS_LIMIT"] = "1"
os.environ['QIANFAN_LLM_API_RETRY_COUNT'] = "3"

## 数据与模板准备

首先构造待评估数据集，此处用本地数据集，也可以拉取千帆平台上的数据集

In [1]:
from qianfan.dataset import Dataset

ds = Dataset.load(data_file="data_summerize/excerpt.jsonl", organize_data_as_group=False, input_columns=["prompt"], reference_column="response")

print(ds[0])

[INFO] [03-19 18:45:32] dataset.py:489 [t:139823724164928]: no data source was provided, construct
[INFO] [03-19 18:45:32] dataset.py:358 [t:139823724164928]: construct a file data source from path: data_summerize/excerpt.jsonl, with args: {'input_columns': ['prompt'], 'reference_column': 'response'}
[INFO] [03-19 18:45:32] file.py:165 [t:139823724164928]: use format type FormatType.Jsonl
[INFO] [03-19 18:45:32] dataset.py:934 [t:139823724164928]: list local dataset data by 0


[{'prompt': '新华社受权于18日全文播发修改后的《中华人民共和国立法法》，修改后的立法法分为“总则”“法律”“行政法规”“地方性法规、自治条例和单行条例、规章”“适用与备案审查”“附则”等6章，共计105条。', 'response': [['修改后的立法法全文公布']]}]


然后创建prompt模版，用于引导模型进行评估。

由于摘要任务没有标准答案，所以此处模板不提供，如果是评估问答任务，则需要提供标准答案以评估正确率。

接着指定评估指标和评估步骤。

此处从四个维度进行评估，分别是相关性、连贯性、一致性、流畅度

In [2]:
from qianfan.common import Prompt

evaluation_prompt_template = """
你是一名裁判员，负责为给定新闻的摘要评分。

评价标准：

{criteria}

请你遵照以下的评分步骤：
{steps}


例子：

新闻：

{prompt}

摘要：

{response}


根据答案的综合水平给出0到{max_score}之间的整数评分。
如果答案存在明显的不合理之处，则应给出一个较低的评分。
如果答案符合以上要求并且与参考答案含义相似，则应给出一个较高的评分

你的回答模版如下:
评分: 此处只能回答整数评分
原因: 此处只能回答评分原因
"""

evaluation_prompt = Prompt(evaluation_prompt_template)

相关性

In [3]:
relevance_metric = """
相关性 - 摘要中重要内容的选择。 \ 
摘要只应包含来自源文档的重要信息。 \
惩罚包含冗余和多余信息的摘要。
"""

relevance_steps = """
1. 仔细阅读摘要和源文档。
2. 将摘要与源文档进行比较，并识别文章的主要观点。
3. 评估摘要是否涵盖了文章的主要观点，以及它包含多少无关或冗余信息。
"""

relevance_max_score = 10

连贯性

In [4]:

coherence_metric = """
连贯性 - 所有句子的总体质量。 \
我们将此维度与 DUC 质量问题结构性和连贯性相关， \
其中“摘要应具有良好的结构和组织，不应只是相关信息的堆叠，而应基于从句子到主题有关的信息的连贯性主体进行构建。” \

"""

coherence_steps = """
1. 仔细阅读文章并识别主要主题和关键点。
2. 阅读摘要并与文章进行比较。检查摘要是否涵盖了文章的主要主题和关键点，并以清晰且逻辑的顺序呈现它们
"""

coherence_max_score = 10

一致性

In [5]:
consistency_metric = """
一致性 - 摘要与摘要源的客观对齐。 \
客观一致的摘要只包含与源文档支持的陈述。 \
惩罚包含幻觉事实的摘要。
"""

consistency_steps = """
1. 仔细阅读文章并识别主要事实和细节。
2. 阅读摘要并与文章进行比较。检查摘要是否包含任何不支持的文章的事实错误。
3. 基于评估标准，为一致性分配分数。
"""

consistency_max_score = 10

流畅度

In [6]:
fluency_metric = """
流畅度：摘要的语法、拼写、标点、单词选择和句子结构的质量。
1：差。摘要有很多错误，使它难以理解或听起来不自然。
2：中。摘要有一些影响文本清晰度或流畅性的错误，但要点仍然可理解。
3：好。摘要很少或没有错误，易于阅读和遵循。
"""

fluency_steps = """
读取摘要并基于给定的标准评估其流畅度。从 1 到 3 分配一个流畅度分数。
"""

fluency_max_score = 3

In [7]:
evaluation_metrics = {
    "Relevance": (relevance_metric, relevance_steps, relevance_max_score),
    "Coherence": (coherence_metric, coherence_steps, coherence_max_score),
    "Consistency": (consistency_metric, consistency_steps, consistency_max_score),
    "Fluency": (fluency_metric, fluency_steps, fluency_max_score),
}

通过千帆sdk，我们有两种方法实现

## 方法一：封装本地评估器

使用千帆平台提供的本地评估工具类进行评估
此方法封装了异步请求，可以根据数据量并发进行评估

首先继承LocalEvaluator类，并实现evaluate方法

In [8]:
import json
from typing import Any, Dict, List, Union, Optional

from qianfan import ChatCompletion
from qianfan.common import Prompt
from qianfan.utils.pydantic import Field
from qianfan.evaluation.evaluator import LocalEvaluator
from qianfan.evaluation.consts import (
    QianfanRefereeEvaluatorPromptTemplate,
    QianfanRefereeEvaluatorDefaultMaxScore,
    QianfanRefereeEvaluatorDefaultMetrics,
    QianfanRefereeEvaluatorDefaultSteps,
)


class LocalJudgeEvaluator(LocalEvaluator):

    model: Optional[ChatCompletion] = Field(default=None, description="model object")
    metric_name: str = Field(default="", description="metric name for evaluation")
    evaluation_prompt: Prompt = Field(default=Prompt(QianfanRefereeEvaluatorPromptTemplate), description="concrete evaluation prompt string")
    prompt_metrics: str = Field(default=QianfanRefereeEvaluatorDefaultMetrics, description="evaluation metrics")
    prompt_steps: str = Field(default=QianfanRefereeEvaluatorDefaultSteps, description="evaluation steps")
    prompt_max_score: int = Field(default=QianfanRefereeEvaluatorDefaultMaxScore, description="max score for evaluation")
    
    class Config:
        arbitrary_types_allowed = True
    def evaluate(
        self, input: Union[str, List[Dict[str, Any]]], reference: str, output: str
    ) -> Dict[str, Any]:
        """
        使用模型进行本地评估
        :param input: 给定的prompt，
            evaluateManager.eval()的is_chat参数为true时,
            input为对话记录，否则为单字符串prompt
        :param reference: 用户给定的标准答案
        :param output: 大模型生成的结果，eval中由service生成，eval_only中由用户给定

        :return: 评估结果
        """
        if isinstance(input, list):
            if not isinstance(self.model, ChatCompletion):
                raise ValueError(f"model is not an instance of ChatCompletion")
            if len(input)!=1:  # 只考虑ChatCompletion单文本输入的情况
                raise ValueError(f"chat history is not single text")
            input_content = input[0].get('content','')
            
            # 生成评价模板
            prompt_text, _ = self.evaluation_prompt.render(
                metric_name=self.metric_name,
                criteria=self.prompt_metrics,
                steps=self.prompt_steps,
                max_score=self.prompt_max_score,
                prompt=input_content,
                response=reference,
            )
            
            # 调用模型获得评分
            msg = qianfan.Messages()
            msg.append(prompt_text)
            
            resp = self.model.do(
                messages=msg,
                temperature=0.1,
                top_p=1,
            )
            
            # print(f'{self.metric_name}|{input[0]}|{output}|{resp["result"].strip()}')
            return {self.metric_name: resp["result"].strip()}
        else:
            raise ValueError(f"input in {type(input)} not supported")

In [9]:
import qianfan
from qianfan.model import Service

eb_turbo_service = Service(model="ERNIE-Bot-turbo")  # 加载生成回答的服务
chat_comp = qianfan.ChatCompletion(model="ERNIE-Bot-4")  # 实例化用于裁判的模型

[WARNING] [03-19 18:45:32] model.py:374 [t:139823724164928]: service type should be specified before exec


附加本地评估器，供EvaluationManager调用

In [10]:
local_evaluators = []
for eval_type, (criteria, steps, max_score) in evaluation_metrics.items():
    local_evaluator = LocalJudgeEvaluator(
        evaluation_prompt=Prompt(evaluation_prompt_template),
        prompt_metrics=criteria,
        prompt_steps=steps,
        prompt_max_score=max_score,
        model=chat_comp,
        metric_name=eval_type
    )
    local_evaluators.append(local_evaluator)

In [11]:
from qianfan.evaluation import EvaluationManager

em = EvaluationManager(local_evaluators=local_evaluators)
result = em.eval(
    [eb_turbo_service], ds,
)

[INFO] [03-19 18:45:32] evaluation_manager.py:420 [t:139823724164928]: start to inference in batch during evaluation
[INFO] [03-19 18:45:32] dataset.py:934 [t:139821972965120]: list local dataset data by None
[INFO] [03-19 18:45:32] openapi_requestor.py:244 [t:139821947754240]: requesting llm api endpoint: /chat/eb-instant
[INFO] [03-19 18:45:32] openapi_requestor.py:244 [t:139821964572416]: requesting llm api endpoint: /chat/eb-instant
[INFO] [03-19 18:45:32] oauth.py:207 [t:139821947754240]: trying to refresh access_token for ak `9FtfEV***`
[INFO] [03-19 18:45:32] openapi_requestor.py:244 [t:139821956163328]: requesting llm api endpoint: /chat/eb-instant
[INFO] [03-19 18:45:32] oauth.py:220 [t:139821947754240]: sucessfully refresh access_token
[INFO] [03-19 18:45:36] base.py:89 [t:139821956163328]: All tasks finished, exeutor will be shutdown
[INFO] [03-19 18:45:36] evaluation_manager.py:444 [t:139823724164928]: start to evaluate llm 0
[INFO] [03-19 18:45:36] openapi_requestor.py:244

In [14]:
result_dataset = result.result_dataset
print(result_dataset.list())

[INFO] [03-19 18:48:27] dataset.py:934 [t:139823724164928]: list local dataset data by None


[{'llm_tag': 'None_None_ERNIE-Bot-turbo', 'input_chats': [{'content': '新华社受权于18日全文播发修改后的《中华人民共和国立法法》，修改后的立法法分为“总则”“法律”“行政法规”“地方性法规、自治条例和单行条例、规章”“适用与备案审查”“附则”等6章，共计105条。', 'role': 'user'}], 'expected_output': '修改后的立法法全文公布', 'llm_output': '《中华人民共和国立法法》是为了规范立法活动，健全国家立法制度，提高立法质量，完善中国特色社会主义法律体系，发挥立法的引领和推动作用，保障和发展社会主义民主，全面推进依法治国，建设社会主义法治国家，根据宪法而制定的。\n\n此次修改后的《中华人民共和国立法法》全文于2023年1月18日由新华社受权播发。全文分为“总则”、“法律”、“行政法规”、“地方性法规、自治条例和单行条例、规章”、“适用与备案审查”、“附则”等6章，共计105条。\n\n这次修改主要集中在完善了立法原则、扩大了全国人大常委会立法权、加强了人大对立法工作的主导作用、健全了法律草案表决程序、完善了授权决定制度、规范了设区的市的立法权等方面。这些修改将有利于发挥立法的引领和推动作用，保障和发展社会主义民主，全面推进依法治国，建设社会主义法治国家。\n\n总的来说，这次修改后的《中华人民共和国立法法》将更好地服务于中国特色社会主义法治体系建设，促进全面依法治国工作的开展。', 'Relevance': '评分：7\n\n原因：摘要“修改后的立法法全文公布”确实捕捉到了新闻文章的核心内容，即《中华人民共和国立法法》已经修改并全文播发。然而，摘要省略了关于立法法结构的具体细节（如分为6章，共计105条），这些信息虽然可能是次要的，但仍然是对主要事件的有益补充。由于摘要简洁地传达了主要信息，但略去了一些细节，因此给予7分的评分。', 'Coherence': '评分: 7\n原因: 摘要确实捕捉到了新闻的主要信息，即“修改后的立法法全文公布”。然而，摘要省略了一些重要的细节，比如立法法被分为6章，共计105条。这些细节对于全面了解新闻内容是有帮助的。尽管如此，摘要还是提供了一个简洁明了的概述，符合其基本目标。在连贯性方面，虽然摘要只有一句话，但

In [15]:
result_df = {"News id":[],"Evaluation Type":[],"Score":[]}
for i, resp in enumerate(result_dataset.list()):
    for metric_name in evaluation_metrics.keys():
        result_df["News id"].append(i)
        # 添加评估类型及对应得分，因为此处只用了一个大模型，所以只取第一个结果
        result_df["Evaluation Type"].append(metric_name)
        score = resp[metric_name].split('\n')[0].replace('：',':').split(':')[-1]
        result_df["Score"].append(score)
    print(resp)

[INFO] [03-19 18:48:27] dataset.py:934 [t:139823724164928]: list local dataset data by None


{'llm_tag': 'None_None_ERNIE-Bot-turbo', 'input_chats': [{'content': '新华社受权于18日全文播发修改后的《中华人民共和国立法法》，修改后的立法法分为“总则”“法律”“行政法规”“地方性法规、自治条例和单行条例、规章”“适用与备案审查”“附则”等6章，共计105条。', 'role': 'user'}], 'expected_output': '修改后的立法法全文公布', 'llm_output': '《中华人民共和国立法法》是为了规范立法活动，健全国家立法制度，提高立法质量，完善中国特色社会主义法律体系，发挥立法的引领和推动作用，保障和发展社会主义民主，全面推进依法治国，建设社会主义法治国家，根据宪法而制定的。\n\n此次修改后的《中华人民共和国立法法》全文于2023年1月18日由新华社受权播发。全文分为“总则”、“法律”、“行政法规”、“地方性法规、自治条例和单行条例、规章”、“适用与备案审查”、“附则”等6章，共计105条。\n\n这次修改主要集中在完善了立法原则、扩大了全国人大常委会立法权、加强了人大对立法工作的主导作用、健全了法律草案表决程序、完善了授权决定制度、规范了设区的市的立法权等方面。这些修改将有利于发挥立法的引领和推动作用，保障和发展社会主义民主，全面推进依法治国，建设社会主义法治国家。\n\n总的来说，这次修改后的《中华人民共和国立法法》将更好地服务于中国特色社会主义法治体系建设，促进全面依法治国工作的开展。', 'Relevance': '评分：7\n\n原因：摘要“修改后的立法法全文公布”确实捕捉到了新闻文章的核心内容，即《中华人民共和国立法法》已经修改并全文播发。然而，摘要省略了关于立法法结构的具体细节（如分为6章，共计105条），这些信息虽然可能是次要的，但仍然是对主要事件的有益补充。由于摘要简洁地传达了主要信息，但略去了一些细节，因此给予7分的评分。', 'Coherence': '评分: 7\n原因: 摘要确实捕捉到了新闻的主要信息，即“修改后的立法法全文公布”。然而，摘要省略了一些重要的细节，比如立法法被分为6章，共计105条。这些细节对于全面了解新闻内容是有帮助的。尽管如此，摘要还是提供了一个简洁明了的概述，符合其基本目标。在连贯性方面，虽然摘要只有一句话，但它

In [16]:
import pandas as pd
pivot_df = pd.DataFrame(result_df, index=None).pivot(
    columns="Evaluation Type", index="News id", values="Score"
).astype(int)
pivot_df['Mean_Score'] = pivot_df.mean(axis=1)
display(pivot_df)

Evaluation Type,Coherence,Consistency,Fluency,Relevance,Mean_Score
News id,,,,,
0,7,8,3,7,6.25
1,7,7,2,7,5.75
2,2,2,1,2,1.75


## 方法二：封装评估函数

除了上述进行本地评估的方法，也可以用ChatCompletion模型构造评估函数，从头开始制作评估的流程

In [17]:
import qianfan

def get_geval_score(chat_comp, evaluation_prompt, **kwargs):
    prompt, _ = evaluation_prompt.render(**kwargs)
    msg = qianfan.Messages()
    msg.append(prompt, role='user')
    resp = chat_comp.do(
        messages=msg,
        temperature=0.1,
        top_p=1,
    )
    return resp['result']

def split_scores_str(judge_result):
    return judge_result.split('\n')[0].replace('评分: ', '').replace('评分：', '')

最后遍历数据集进行评估

In [18]:
chat_comp = qianfan.ChatCompletion(model="ERNIE-Bot-4")
result = {"Evaluation Type": [], "News id": [], "Score": []}
for eval_type, (criteria, steps, max_score) in evaluation_metrics.items():
    for ind, data in enumerate(ds.list()):
        result["Evaluation Type"].append(eval_type)
        result["News id"].append(ind)
        evaluate = get_geval_score(
            chat_comp,
            evaluation_prompt,
            criteria=criteria,
            steps=steps,
            prompt=data[0]['prompt'],
            response=data[0]['response'][0][0],
            max_score=str(max_score),
            metric_name=eval_type
        )
        score = int(split_scores_str(evaluate.strip()))
        result["Score"].append(score)


[INFO] [03-19 18:48:27] dataset.py:934 [t:139823724164928]: list local dataset data by None
[INFO] [03-19 18:48:27] openapi_requestor.py:244 [t:139823724164928]: requesting llm api endpoint: /chat/completions_pro
[INFO] [03-19 18:48:36] openapi_requestor.py:244 [t:139823724164928]: requesting llm api endpoint: /chat/completions_pro
[INFO] [03-19 18:48:54] openapi_requestor.py:244 [t:139823724164928]: requesting llm api endpoint: /chat/completions_pro
[INFO] [03-19 18:49:00] dataset.py:934 [t:139823724164928]: list local dataset data by None
[INFO] [03-19 18:49:00] openapi_requestor.py:244 [t:139823724164928]: requesting llm api endpoint: /chat/completions_pro
[INFO] [03-19 18:49:11] openapi_requestor.py:244 [t:139823724164928]: requesting llm api endpoint: /chat/completions_pro
[INFO] [03-19 18:49:22] openapi_requestor.py:244 [t:139823724164928]: requesting llm api endpoint: /chat/completions_pro
[INFO] [03-19 18:49:36] dataset.py:934 [t:139823724164928]: list local dataset data by Non

In [19]:
print(result)

{'Evaluation Type': ['Relevance', 'Relevance', 'Relevance', 'Coherence', 'Coherence', 'Coherence', 'Consistency', 'Consistency', 'Consistency', 'Fluency', 'Fluency', 'Fluency'], 'News id': [0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2], 'Score': [8, 7, 2, 7, 6, 2, 8, 7, 2, 3, 2, 1]}


查看最终结果

In [20]:
pivot_df = pd.DataFrame(result, index=None).pivot(
    columns="Evaluation Type", index="News id", values="Score"
).astype(int)
pivot_df['Mean_Score'] = pivot_df.mean(axis=1)
display(pivot_df)

Evaluation Type,Coherence,Consistency,Fluency,Relevance,Mean_Score
News id,,,,,
0,7,8,3,8,6.50
1,6,7,2,7,5.50
2,2,2,1,2,1.75
